In [6]:
# import libs
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB


In [9]:
# format 
%matplotlib inline

# execute data from comments
def obter_dados_das_fontes():
    dados = pd.read_csv('wiki.csv')
    comment = dados['Comment']
    evaluation = dados['Evaluation']
    return dados 

print (obter_dados_das_fontes())

print("O conjunto de dados dos comentarios {} pontos com {} variáveis em cada.".format(*obter_dados_das_fontes().shape))

                                                Comment  Evaluation
0     Um dia triste para o Brasil  o crime compensa ...         0.0
1                   Liberdade para dentro da cabeça....         1.0
2                                   AI 5 AGORAAAAAAA!!!         0.0
3     a unica coisa boa nisso vai ser ver ele sendo ...         0.0
4                             Esse país é uma vergonha.         0.0
...                                                 ...         ...
1039  Ele está enquadrado na Lei da Ficha Limpa. Mes...         NaN
1040                                             Demorô         NaN
1041  Justica foi feita. Vamos ao levante acabar com...         NaN
1042                       Soltem o Barbudo Bandido!!!          NaN
1043  Gente fechem as portas da sua casa Lulaladrão ...         NaN

[1044 rows x 2 columns]
O conjunto de dados dos comentarios 1044 pontos com 2 variáveis em cada.


In [10]:
def exibir_resultado(valor):
    frase, resultado = valor
    resultado = "Frase positiva" if resultado[0] == '1' else "Frase negativa"
    print(frase, ":", resultado)
    
def analisar_frase(classificador, vetorizador, frase):
    return frase, classificador.predict(vetorizador.transform([frase]))    

# divido os dados para treino e validação
def dividir_dados_para_treino_e_validacao(dados):
    quantidade_total = len(dados)
    percentual_para_treino = 200/quantidade_total
    treino = []
    validacao = []

    for indice in range(0, quantidade_total):
        if indice < quantidade_total * percentual_para_treino:
            treino.append((dados['Comment'][indice],dados['Evaluation'][indice]))
        else:
            validacao.append((dados['Comment'][indice],dados['Evaluation'][indice]))

    return treino, validacao

# separar dados que não são bons
def pre_processamento():
    dados = obter_dados_das_fontes()
#     dados_tratados = tratamento_dos_dados(dados)
    
    return dividir_dados_para_treino_e_validacao(dados)

# realizo  treinamento dos dados para treinamento; separo em 0 e 1
def realizar_treinamento(registros_de_treino, vetorizador):
    treino_comentarios = [registro_treino[0] for registro_treino in registros_de_treino]
    treino_respostas = [registro_treino[1] for registro_treino in registros_de_treino]

    treino_comentarios = vetorizador.fit_transform(treino_comentarios)

    return BernoulliNB().fit(treino_comentarios, treino_respostas)

# availiaçõ do treinamento feito
def realizar_avaliacao_simples(registros_para_avaliacao):
    avaliacao_comentarios = [registro_avaliacao[0] for registro_avaliacao in registros_para_avaliacao]
    avaliacao_respostas   = [registro_avaliacao[1] for registro_avaliacao in registros_para_avaliacao]

    total = len(avaliacao_comentarios)
    acertos = 0
    for indice in range(0, total):
        resultado_analise = analisar_frase(classificador, vetorizador, avaliacao_comentarios[indice])
        frase, resultado = resultado_analise
        acertos += 1 if resultado[0] == avaliacao_respostas[indice] else 0

    return acertos * 100 / total

def realizar_avaliacao_completa(registros_para_avaliacao):
    avaliacao_comentarios = [registro_avaliacao[0] for registro_avaliacao in registros_para_avaliacao]
    avaliacao_respostas   = [registro_avaliacao[1] for registro_avaliacao in registros_para_avaliacao]

    total = len(avaliacao_comentarios)
    verdadeiros_positivos = 0
    verdadeiros_negativos = 0
    falsos_positivos = 0
    falsos_negativos = 0

    for indice in range(0, total):
        resultado_analise = analisar_frase(classificador, vetorizador, avaliacao_comentarios[indice])
        frase, resultado = resultado_analise
        if resultado[0] == '0':
            verdadeiros_negativos += 1 if avaliacao_respostas[indice] == '0' else 0
            falsos_negativos += 1 if avaliacao_respostas[indice] != '0' else 0
        else:
            verdadeiros_positivos += 1 if avaliacao_respostas[indice] == '1' else 0
            falsos_positivos += 1 if avaliacao_respostas[indice] != '1' else 0

        return ( verdadeiros_positivos * 100 / total, 
             verdadeiros_negativos * 100 / total,
             falsos_positivos * 100 / total,
             falsos_negativos * 100 / total
           )
            
registros_de_treino, registros_para_avaliacao = pre_processamento()
vetorizador = CountVectorizer(binary = 'true')
classificador = realizar_treinamento(registros_de_treino, vetorizador)

exibir_resultado( analisar_frase(classificador, vetorizador,"Liberdade para dentro da cabeça...."))
exibir_resultado( analisar_frase(classificador, vetorizador,"Lula lindo"))
exibir_resultado( analisar_frase(classificador, vetorizador,"VIVA LADROES"))
exibir_resultado( analisar_frase(classificador, vetorizador,"pais nojento stf comprado"))
exibir_resultado( analisar_frase(classificador, vetorizador,"barbudo mais carismático do país"))


percentual_acerto = realizar_avaliacao_simples(registros_para_avaliacao)
informacoes_analise = realizar_avaliacao_completa(registros_para_avaliacao)

verdadeiros_positivos,verdadeiros_negativos,falsos_positivos,falsos_negativos = informacoes_analise

print("O modelo teve uma taxa de acerto de", percentual_acerto, "%")

# print("Onde", verdadeiros_positivos, "% são verdadeiros positivos")
# print("e", verdadeiros_negativos, "% são verdadeiros negativos")

# print("e", falsos_positivos, "% são falsos positivos")
# print("e", falsos_negativos, "% são falsos negativos")

Liberdade para dentro da cabeça.... : Frase negativa
Lula lindo : Frase negativa
VIVA LADROES : Frase negativa
pais nojento stf comprado : Frase negativa
barbudo mais carismático do país : Frase negativa
O modelo teve uma taxa de acerto de 0.47393364928909953 %
